In [1]:
from pprint import pprint

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW
import torch.nn.functional as F
import torch.utils.data as data


from src.params import ModelName, DatasetName, DeviceType, get_new_seed
from src.models import LogisticRegression
from src.datasets import load_embeddings
from src.datasets import DatasetSampler
from src.engine import DatasetTrainer, DatasetInference
from src.metrics import cross_entropy, reverse_loss, logistic_bregman
from src.measurements import EstimateKLMisfit
from src.np_utils import index_random_split

# Setup

## Models

Weak Supervisor = AlexNet

Strong Model 1 = Dino ResNet50
Strong Model 2 = Dino Vit-B/8

## Datasets

1. Cifar10
2. ImageNet (download in progress)
3. Cifar100?

For now, not caring about whether validation set is same across models

In [2]:
weak_model_id = ModelName.ALEXNET
strong_model_id = ModelName.RESNET50_DINO

dataset_id = DatasetName.CIFAR10

In [3]:
# Load the data
weak_embeddings = load_embeddings(model_name=weak_model_id, dataset_name=dataset_id)

In [4]:
# Some sanity checks
assert weak_embeddings.x_train.shape[0] == weak_embeddings.y_train.shape[0]
assert weak_embeddings.x_train.shape[1] == weak_embeddings.x_test.shape[1]

# Print some stats on the data
print("Weak model embeddings:")
print("Train data shape:", weak_embeddings.x_train.shape)
print("Test data shape:", weak_embeddings.x_test.shape)
print("Number of classes:", weak_embeddings.num_classes)
# Dtypes
print("Train data dtype:", weak_embeddings.x_train.dtype)
print("Test data dtype:", weak_embeddings.x_test.dtype)
print("Train Labels dtype:", weak_embeddings.y_train.dtype)
print("Test Labels dtype:", weak_embeddings.y_test.dtype)


Weak model embeddings:
Train data shape: (50000, 9216)
Test data shape: (10000, 9216)
Number of classes: 10
Train data dtype: float32
Test data dtype: float32
Train Labels dtype: int64
Test Labels dtype: int64


In [5]:
weak_split = 0.3
validation_split = 0.2
validation_splits = [validation_split * weak_split, (1 - validation_split) * weak_split, validation_split * (1 - weak_split), (1 - validation_split) * (1 - weak_split)]    
weak_val_inds, weak_train_inds, st_from_wk_val_inds, st_from_wk_train_inds = index_random_split(weak_embeddings.x_train.shape[0], split_sizes=validation_splits, random_state=np.random.default_rng(seed=get_new_seed()))

In [6]:
weak_train_dataset = data.TensorDataset(
    torch.tensor(weak_embeddings.x_train[weak_train_inds]),
    F.one_hot(
        torch.tensor(weak_embeddings.y_train[weak_train_inds]), num_classes=weak_embeddings.num_classes
    ).to(torch.float32),
)
weak_val_dataset = data.TensorDataset(
    torch.tensor(weak_embeddings.x_train[weak_val_inds]),
    F.one_hot(
        torch.tensor(weak_embeddings.y_train[weak_val_inds]), num_classes=weak_embeddings.num_classes
    ).to(torch.float32),
)

In [8]:
import math

# Some sanity checks
# assert len(weak_train_dataset) + len(weak_val_dataset) + len(st_gt_val_inds) + len(st_from_wk_val_inds)== weak_embeddings.x_train.shape[0]
assert len(weak_val_dataset) == int(validation_splits[0] * weak_embeddings.x_train.shape[0])
assert len(weak_train_dataset) == int(validation_splits[1] * weak_embeddings.x_train.shape[0])

In [9]:
# Instantiate the weak_model 
num_inputs = weak_embeddings.x_train.shape[1]
weak_model = LogisticRegression(num_inputs, weak_embeddings.num_classes)


In [10]:
# Train the weak model
weight_decay = 0.0
lr = 1e-3
num_epochs = 20
batch_size=256
validation_split=0.2

n_iter = num_epochs * len(range(0, len(weak_train_dataset), batch_size))

optimizer = optim.Adam(weak_model.parameters(), weight_decay=weight_decay, lr=lr)
schedule = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=n_iter)

weak_trainer = DatasetTrainer(
    model=weak_model,
    optimizer=optimizer,
    loss_fn=cross_entropy,
    dataset=weak_train_dataset,
    val_dataset=weak_val_dataset,
    scheduler=schedule,
)
weak_trainer.train(num_epochs=num_epochs, batch_size=batch_size,)

Using model device for training: cpu
Using model device for training: cpu
Epoch 1/20


100%|██████████| 47/47 [00:00<00:00, 164.00it/s, loss=0.665]


Validating model...


100%|██████████| 12/12 [00:00<00:00, 312.55it/s]


Validation loss: 0.6511116971572241
Epoch 2/20


...

100%|██████████| 47/47 [00:00<00:00, 182.98it/s, loss=0.0507]


Validating model...


100%|██████████| 12/12 [00:00<00:00, 267.61it/s]

Validation loss: 0.6133474260568619
Best model found at epoch 4 with val loss 0.5671936323245367



/home/abhmul/dev/pythagoras-agi-new/src/engine/trainer.py:242: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.validation_tmp_path)

In [11]:
weak_full_dataset = data.TensorDataset(
    torch.tensor(weak_embeddings.x_train),
    F.one_hot(
        torch.tensor(weak_embeddings.y_train), num_classes=weak_embeddings.num_classes
    ).to(torch.float32),
)

weak_inference = DatasetInference(
    weak_model,
    weak_full_dataset,
)
weak_logits = weak_inference.inference(batch_size=batch_size)
weak_labels = torch.softmax(weak_logits, dim=1)

Using model device for inference: cpu


100%|██████████| 196/196 [00:00<00:00, 300.72it/s]


In [12]:
weak_embeddings = None
weak_test_dataset = None
weak_train_dataset = None
weak_val_dataset = None
weak_trainer = None

In [13]:
# Load the data
strong_embeddings = load_embeddings(model_name=strong_model_id, dataset_name=dataset_id)

# # Regenerate the split to see if this changes things?
# validation_split = 0.2
# strong_gt_val_inds, strong_gt_train_inds = index_random_split(strong_embeddings.x_train.shape[0], split_sizes=[validation_split, 1-validation_split], random_state=np.random.default_rng(seed=get_new_seed()))

strong_train_dataset = data.TensorDataset(
    torch.tensor(strong_embeddings.x_train[st_from_wk_train_inds]),
    F.one_hot(
        torch.tensor(strong_embeddings.y_train[st_from_wk_train_inds]), num_classes=strong_embeddings.num_classes
    ).to(torch.float32),
)
strong_val_dataset = data.TensorDataset(
    torch.tensor(strong_embeddings.x_train[st_from_wk_val_inds]),
    F.one_hot(
        torch.tensor(strong_embeddings.y_train[st_from_wk_val_inds]), num_classes=strong_embeddings.num_classes
    ).to(torch.float32),
)

In [14]:
# Instantiate the strong 
num_inputs = strong_embeddings.x_train.shape[1]
strong_gt_model = LogisticRegression(num_inputs, strong_embeddings.num_classes)

In [15]:
# Train the strong ground-truth model
weight_decay = 0.0
lr = 1e-3
num_epochs = 200
batch_size=256

n_iter = num_epochs * len(range(0, len(strong_train_dataset), batch_size))

optimizer = optim.Adam(strong_gt_model.parameters(), weight_decay=weight_decay, lr=lr)
schedule = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=n_iter)

strong_trainer = DatasetTrainer(
    model=strong_gt_model,
    optimizer=optimizer,
    loss_fn=cross_entropy,
    dataset=strong_train_dataset,
    val_dataset=strong_val_dataset,
    scheduler=schedule,
)
strong_trainer.train(num_epochs=num_epochs, batch_size=batch_size, log_every=10)

Using model device for training: cpu
Using model device for training: cpu
Epoch 1/200


100%|██████████| 110/110 [00:00<00:00, 337.75it/s, loss=1.08]


Validating model...


100%|██████████| 28/28 [00:00<00:00, 863.50it/s]


Validation loss: 1.0642236002853938
Epoch 11/200


...

Validation loss: 0.27999495023063253
Epoch 191/200


100%|██████████| 110/110 [00:00<00:00, 371.03it/s, loss=0.147]


Validating model...


100%|██████████| 28/28 [00:00<00:00, 541.74it/s]


Validation loss: 0.27998711221984457
Best model found at epoch 127 with val loss 0.27938142738171984


In [16]:
# validation_split = 0.2
# val_inds, train_inds = index_random_split(strong_embeddings.x_train.shape[0], split_size=validation_split, random_state=np.random.default_rng(seed=get_new_seed()))

weakly_labelled_train_dataset = data.TensorDataset(
    torch.tensor(strong_embeddings.x_train[st_from_wk_train_inds]),
    weak_labels[st_from_wk_train_inds],
    weak_logits[st_from_wk_train_inds],
)
weakly_labelled_val_dataset = data.TensorDataset(
    torch.tensor(strong_embeddings.x_train[st_from_wk_val_inds]),
    weak_labels[st_from_wk_val_inds],
    weak_logits[st_from_wk_val_inds],
)

In [17]:
# Instantiate the strong from weak
num_inputs = strong_embeddings.x_train.shape[1]
strong_from_wk_model = LogisticRegression(num_inputs, strong_embeddings.num_classes)

In [18]:
# Train the strong model from weak
weight_decay = 0.0
lr = 1e-3
num_epochs = 200
batch_size = 256

n_iter = num_epochs * len(range(0, len(weakly_labelled_train_dataset), batch_size))

optimizer = optim.Adam(
    strong_from_wk_model.parameters(), weight_decay=weight_decay, lr=lr
)
schedule = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=n_iter)

strong_from_wk_trainer = DatasetTrainer(
    model=strong_from_wk_model,
    optimizer=optimizer,
    loss_fn=reverse_loss(logistic_bregman),
    dataset=weakly_labelled_train_dataset,
    val_dataset=weakly_labelled_val_dataset,
    scheduler=schedule,
    use_label_logits=True,
)
strong_from_wk_trainer.train(
    num_epochs=num_epochs, batch_size=batch_size, log_every=10
)

Using model device for training: cpu
Using model device for training: cpu
Epoch 1/200


100%|██████████| 110/110 [00:00<00:00, 253.34it/s, loss=2.03]


Validating model...


100%|██████████| 28/28 [00:00<00:00, 261.30it/s]


Validation loss: 1.8740744037287576
Epoch 11/200


...

Validation loss: 0.35366862054382053
Epoch 191/200


100%|██████████| 110/110 [00:00<00:00, 228.86it/s, loss=0.222]


Validating model...


100%|██████████| 28/28 [00:00<00:00, 631.29it/s]


Validation loss: 0.35368819002594265
Best model found at epoch 73 with val loss 0.3496598643915994


In [19]:
weak_embeddings = load_embeddings(model_name=weak_model_id, dataset_name=dataset_id)
strong_embeddings = load_embeddings(model_name=strong_model_id, dataset_name=dataset_id)
estimate_misfit_dataset = data.TensorDataset(
    torch.tensor(strong_embeddings.x_test),
    torch.tensor(weak_embeddings.x_test),
    F.one_hot(torch.tensor(weak_embeddings.y_test), num_classes=weak_embeddings.num_classes).to(torch.float32),
)

In [20]:
misfit_estimator = EstimateKLMisfit(
    strong_model=strong_from_wk_model,
    weak_model=weak_model,
    strong_gt_model=strong_gt_model,
    dataset=estimate_misfit_dataset,
)
result = misfit_estimator.estimate(
    batch_size=256, device=torch.device(DeviceType.CPU.value)
)
pprint(result)

100%|██████████| 40/40 [00:00<00:00, 128.24it/s, gt_to_st_xe=0.424, gt_to_wk_xe=0.571, stgt_to_st_xe=0.43, stgt_to_wk_xe=0.735, stgt_to_st=0.144, stgt_to_wk=0.449, st_to_wk=0.349, wk_to_st=0.396, gain_xe=0.148, misfit_xe_error=-0.202, stgt_gain_xe=0.306, stgt_misfit_xe_error=-0.0434, stgt_gain=0.306, stgt_misfit_error=-0.0434, st_ent=0.444, wk_ent=0.483, stgt_ent=0.286] 

{'gain_xe__err': 0.017217382788658142,
 'gain_xe__mean': 0.14752988517284393,
 'gain_xe__std': 0.8608691096305847,
 'gt_to_st_xe__err': 0.016732661053538322,
 'gt_to_st_xe__mean': 0.42387655377388,
 'gt_to_st_xe__std': 0.836633026599884,
 'gt_to_wk_xe__err': 0.021549470722675323,
 'gt_to_wk_xe__mean': 0.5714066624641418,
 'gt_to_wk_xe__std': 1.077473521232605,
 'misfit_xe_error__err': 0.01751331053674221,
 'misfit_xe_error__mean': -0.20164009928703308,
 'misfit_xe_error__std': 0.8756654858589172,
 'st_ent__err': 0.008803303353488445,
 'st_ent__mean': 0.4440767168998718,
 'st_ent__std': 0.4401651620864868,
 'st_to_wk__err': 0.011510785669088364,
 'st_to_wk__mean': 0.34917014837265015,
 'st_to_wk__std': 0.5755392909049988,
 'stgt_ent__err': 0.0075825853273272514,
 'stgt_ent__mean': 0.2859863340854645,
 'stgt_ent__std': 0.3791292607784271,
 'stgt_gain__err': 0.013827413320541382,
 'stgt_gain__mean': 0.3057228922843933,
 'stgt_gain__std': 0.6913706660270691,
 'stgt_gain_xe__err': 0.0138274